In [ ]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"

import json
import random
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from PIL import Image, ImageEnhance
from typing import Optional

import trimesh
import pyrender
from pyrender.constants import RenderFlags
from tqdm.notebook import tqdm
import maxflow

## Utils

In [ ]:
def compute_pose(img: np.ndarray):
    img_size = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_corners, img_ids, rejected_corners = detector.detectMarkers(gray)
    
    img_corners, img_ids, rejected_corners, recovered_ids = detector.refineDetectedMarkers(
        img,
        board,
        img_corners,
        img_ids,
        rejected_corners
    )
    
    obj_points, img_points = board.matchImagePoints(img_corners, img_ids)
    
    retval, rvec, tvec = cv2.solvePnP(
        obj_points,
        img_points,
        cameraMatrix,
        distCoeffs
    )

    rvec, tvec = cv2.solvePnPRefineLM(
        obj_points,
        img_points,
        cameraMatrix,
        distCoeffs,
        rvec,
        tvec
    )
    
    R_oc, _ = cv2.Rodrigues(rvec)
    R_wc = T @ R_oc.T
    t_wc = -R_wc @ tvec
    return R_wc, t_wc, rvec, tvec

def rgb_to_hex(rgb):
    return '#{:02X}{:02X}{:02X}'.format(*rgb)

## Scene Generation

In [ ]:
def norm(v: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

class Scene:
    def __init__(self, mesh, image_width: int = 640, image_height: int = 480) -> None:

        self.image_width = image_width
        self.image_height = image_height
        
        naruto_mesh = pyrender.Mesh.from_trimesh(mesh)
        naruto_pose = np.eye(4)
        naruto_pose[1, 3] = -1
        
        self.pyrender_scene = pyrender.Scene(ambient_light=[0.5, 0.5, 0.5])
        self.pyrender_scene.add(naruto_mesh, pose=naruto_pose)
        
        self.renderer = pyrender.OffscreenRenderer(image_width, image_height)
        camera = pyrender.PerspectiveCamera(yfov = np.pi / 3.0, aspectRatio=1.0)
        self.camera_node = pyrender.Node(camera=camera)
        self.pyrender_scene.add_node(self.camera_node)

        self.point_clouds = []

        self.T = np.array([
            [1, 0, 0],
            [0, -1, 0],
            [0, 0, -1]
        ])
        
    def add_points(self, points: np.ndarray, radius: float = 0.01, color: list[float] = [0, 0, 0.5]) -> None:
        sm = trimesh.creation.uv_sphere(radius=radius)
        sm.visual.vertex_colors = color
        tfs = np.tile(np.eye(4), (len(points), 1, 1))
        tfs[:,:3,3] = points
        pts = pyrender.Mesh.from_trimesh(sm, poses=tfs)
        node = self.pyrender_scene.add(pts)
        self.point_clouds.append(node)

    def remove_all_points(self) -> None:
        for ptsc in self.point_clouds:
            self.pyrender_scene.remove_node(ptsc)
        self.point_clouds = []
        
    def sample(self, alpha: Optional[float] = None, beta: Optional[float] = None, gamma: Optional[float] = None, return_depth: bool = False) -> [Image.Image, np.ndarray, np.ndarray]:
        rand = np.random.uniform(0, 360, (3)) * np.pi / 180
        if alpha == None:
            alpha = rand[0]
        else:
            alpha *= np.pi / 180
        if beta == None:
            beta = rand[1]
        else:
            beta *= np.pi / 180
        if gamma ==  None:
            gamma = rand[2]
        else:
            gamma *= np.pi / 180
        Rx = np.array([
                            [1, 0, 0, 0],
                            [0, np.cos(alpha), -np.sin(alpha), 0],
                            [0, np.sin(alpha), np.cos(alpha), 0],
                            [0, 0, 0, 1]]
                        )
        Ry = np.array([
                            [np.cos(beta), 0, np.sin(beta), 0],
                            [0, 1, 0, 0],
                            [-np.sin(beta), 0, np.cos(beta), 0],
                            [0, 0, 0, 1]]
                        )
        Rz = np.array([
                            [np.cos(gamma), -np.sin(gamma), 0, 0],
                            [np.sin(gamma), np.cos(gamma), 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]]
                        )
        camera_pose = Rx @ Ry @ Rz @ np.array([
                            [1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 1, 2],
                            [0, 0, 0, 1]]
                        )
        self.pyrender_scene.set_pose(self.camera_node, camera_pose)
        color, depth = self.renderer.render(self.pyrender_scene)
        image = Image.fromarray(color)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(2.0)
        if not return_depth:
            depth[depth > 0] = 1
        return image, camera_pose, depth

    def generate_samples(self, n_samples: int):
        images = []
        poses = []
        masks = []
        for _ in range(n_samples):
            image, camera_pose, depth = self.sample()
            images.append(image)
            poses.append(camera_pose)
            masks.append(depth)
        return images, poses, masks

    def intrinsics_matrix(self):

        P = self.camera_node.camera.get_projection_matrix()
        fx = P[0, 0] * self.image_width / 2
        fy = P[1, 1] * self.image_height / 2
        cx = (1.0 - P[0, 2]) * self.image_width / 2
        cy = (1.0 + P[1, 2]) * self.image_height / 2
    
        K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    
        return K

mesh = trimesh.load("./Naruto/Naruto.obj", force="mesh")
scene = Scene(mesh = mesh)
images, poses, masks = scene.generate_samples(n_samples=25)

## Generating Visual Hull

In [ ]:
reso = 300

K = scene.intrinsics_matrix()

_fx = K[0, 0]
_fy = K[1, 1]
_cx = K[0, 2]
_cy = K[1, 2]

cam_pos = np.array([0, 0, 2])
bounds = np.array([
    [-0.810964,  0.001889, -0.810964],
    [ 0.810959,  1.7243  ,  0.810959]
])
bounds[:, 1] -= 1
n_vxls = np.array([reso, reso, reso])
volume = np.zeros(n_vxls, dtype=np.uint8)
intensities = np.zeros((*list(n_vxls), 3), dtype=np.float16)
step = (bounds[1] - bounds[0]) / n_vxls

xi = np.arange(n_vxls[0])
yi = np.arange(n_vxls[1])
zi = np.arange(n_vxls[2])
indices = np.stack(np.meshgrid(xi, yi, zi, indexing='ij'), axis=-1)

centers = bounds[0] + step / 2.0 + step * indices
centers = centers.reshape(-1, 3)

for image, pose, mask in tqdm(zip(images, poses, masks), total=len(images)):
    
    [x, y, z] = scene.T @ np.linalg.inv(pose[:3, :3]) @ centers.T + cam_pos[:, None]
    x_proj = (_cx + np.rint(x / z * _fx)).astype(np.int32)
    y_proj = (_cy + np.rint(y / z * _fy)).astype(np.int32)
    
    H, W = mask.shape
    valid = (x_proj >= 0) & (x_proj < W) & (y_proj >= 0) & (y_proj < H)
    x_proj = np.clip(x_proj, 0, W-1)
    y_proj = np.clip(y_proj, 0, H-1)
    
    valid_voxels = (valid & (mask[y_proj, x_proj] > 0)).reshape(n_vxls)
    voxel_indices = indices[valid_voxels]
    
    x_idx, y_idx, z_idx = voxel_indices.T
    volume[x_idx, y_idx, z_idx] += 1
    I = (np.array(image)[y_proj, x_proj].astype(np.float16) * valid.astype(np.float16)[:, None]).reshape((*list(n_vxls), 3))
    intensities += I

intensities /= len(images)
intensities = np.clip(np.rint(intensities), a_min=0, a_max=255).astype(int)

obj_mask = volume >= len(images)

data = {
    "version": "Voxel Builder 4.5.1",
    "project": {
        "name": "naruto",
        "voxels": int(np.sum(obj_mask))
    },
    "data": {
        "voxels": ";".join([f"{v[0]},{v[1]},{v[2]},#EE4B2B,1" for v in zip(*np.where(obj_mask))])
    }
}
with open('model.json', 'w') as fp:
    json.dump(data, fp)

In [ ]:
new_volume = np.zeros_like(volume, dtype=np.uint8)
offsets = np.array([
    [dx, dy, dz]
    for dx in [-1, 0, 1]
    for dy in [-1, 0, 1]
    for dz in [-1, 0, 1]
])
for x, y, z in zip(*np.where(volume >= len(images))):
    new_volume[tuple(np.clip(np.array([x, y, z]) + offsets, a_min=0, a_max=reso-1).T)] = 1
data = {
    "version": "Voxel Builder 4.5.1",
    "project": {
        "name": "naruto",
        "voxels": int(np.sum(new_volume))
    },
    "data": {
        "voxels": ";".join([
            f"{v[0]},{v[1]},{v[2]},#EE4B2B,1"
            for v in zip(*np.where(new_volume > 0))
        ])
    }
}
with open('model.json', 'w') as fp:
    json.dump(data, fp)

## Refining the Visual Hull

ref: [Multi View Stereo via Volumetric Graph Cut](https://www.cs.jhu.edu/~misha/Fall13b/Papers/Vogiatzis05.pdf)

In [ ]:
class VolumetricGraph:
    def __init__(self, volume, centers, intensities, voxel_size, _lambda=1e6):
        self.volume = volume
        self.centers = centers
        self.intensities = intensities
        self.voxel_size = voxel_size
        self._lambda = _lambda

        print("Initializing Hull Surface")
        self.init_hull_surf()
        print("Initializing Graph")
        self.init_graph()

    def init_hull_surf(self):
        self.hull_surf = np.zeros_like(self.volume)
        nx, ny, nz = self.volume.shape
        for x in range(nx):
            for y in range(ny):
                for z in range(nz):
                    if self.volume[x, y, z] > 0:
                        self.hull_surf[x, y, z] = 1
                        if x > 0 and x < nx - 1 and (self.volume[x-1, y, z] == 0 or self.volume[x+1, y, z] == 0):
                            self.hull_surf[x, y, z] = 2
                        if y > 0 and y < ny -1 and (self.volume[x, y-1, z] == 0 or self.volume[x, y+1, z] == 0):
                            self.hull_surf[x, y, z] = 2
                        if z > 0 and z < nz -1 and (self.volume[x, y, z-1] == 0 or self.volume[x, y, z+1] == 0):
                            self.hull_surf[x, y, z] = 2

    def init_graph(self):
        self.graph = maxflow.Graph[float]()
        self.node_ids = self.graph.add_grid_nodes(self.volume.shape)
        self.add_t_links()
        self.add_n_links()

    def add_t_links(self):
        sink_inds = np.where(self.hull_surf == 2)
        sink_nodes = self.node_ids[sink_inds]
        for node in sink_nodes.flatten():
            self.graph.add_tedge(node, 0, float('inf'))
        source_inds = np.where(self.hull_surf == 1)
        source_nodes = self.node_ids[source_inds]
        Wb = self._lambda * np.prod(self.voxel_size)
        for node in source_nodes.flatten():
            self.graph.add_tedge(node, Wb, 0)

    def add_n_links(self):
        nx, ny, nz = self.volume.shape
        sigma = 30

        for x in range(nx):
            for y in range(ny):
                for z in range(nz):
                    if self.volume[x, y, z] > 0:
                        node = self.node_ids[x, y, z]
                        I_i = self.intensities[x, y, z, :]
                        for dx, dy, dz in [(-1, 0, 0), (0, -1, 0), (0, 0, -1)]:
                            nx_ = x + dx
                            ny_ = y + dy
                            nz_ = z + dz
                            if nx_ >= 0 and ny_ >= 0 and nz_ >= 0 and self.volume[nx_, ny_, nz_] > 0:
                                neighbor_node = self.node_ids[nx_, ny_, nz_]
                                I_j = self.intensities[nx_, ny_, nz_, :]
                                D_ij = np.linalg.norm(I_i - I_j) ** 2
                                W = np.exp(- D_ij / (2 * sigma ** 2))
                                self.graph.add_edge(node, neighbor_node, W, W)

    def run(self):
        print("Computing Graph Cut")
        self.graph.maxflow()
        self.segmentation = self.graph.get_grid_segments(self.node_ids)

    def get_refined_volume(self):
        return self.segmentation.astype(np.uint8)

vg = VolumetricGraph(
    volume = volume,
    centers = centers,
    intensities = intensities,
    voxel_size = step,
    _lambda = 1e6
)
vg.run()
refined_volume = vg.get_refined_volume()

data = {
    "version": "Voxel Builder 4.5.1",
    "project": {
        "name": "naruto",
        "voxels": int(np.sum(refined_volume))
    },
    "data": {
        "voxels": ";".join([f"{v[0]},{v[1]},{v[2]},#EE4B2B,1" for v in zip(*np.where(refined_volume))])
    }
}
with open('refined_model.json', 'w') as fp:
    json.dump(data, fp)

## Shading the Object

In [ ]:
intensities = np.zeros((*list(n_vxls), 4), dtype=np.float32)

for image, pose, mask in tqdm(zip(images, poses, masks), total=len(images)):
    image = np.array(image).astype(np.float32)
    mask = np.array(mask)
    seg = np.where(mask > 0)
    y, x = seg[0], seg[1]

    dx = (x.astype(np.float32) - _cx) / _fx
    dy = (y.astype(np.float32) - _cy) / _fy
    dz = np.ones_like(dx, dtype=np.float32)

    rays = (pose[:3, :3] @ scene.T @ np.vstack((dx, dy, dz))).T
    casting = np.ones(len(dx), dtype=bool)

    for z in np.arange(2000) * step.min() / 2:

        points = pose[:3, :3] @ cam_pos + rays * z

        voxel_coords = np.floor((points - bounds[0]) / step).astype(int)

        valid_mask = ((voxel_coords >= 0) & (voxel_coords < n_vxls)).all(axis=1)
        valid_voxels = voxel_coords[valid_mask]
        valid_indices = np.where(valid_mask)[0]

        if len(valid_voxels) == 0:
            continue


        surface_voxels = volume[valid_voxels[:, 0], valid_voxels[:, 1], valid_voxels[:, 2]] >= len(images)
        surf_voxel_indices = valid_indices[surface_voxels]

        if len(surf_voxel_indices) > 0:
            sv_coords = valid_voxels[surface_voxels]
            colors = image[y[surf_voxel_indices], x[surf_voxel_indices]]
            for idx, coord in enumerate(sv_coords):
                i, j, k = coord
                intensities[i, j, k, :3] += colors[idx]
                intensities[i, j, k, 3] += 1

        casting[valid_indices] &= ~surface_voxels
        if not casting.any():
            break

non_zero_counts = intensities[..., 3] > 0
intensities[non_zero_counts, :3] /= intensities[non_zero_counts, 3, None]

intensities = np.clip(np.rint(intensities[:, :, :, :3]), a_min=0, a_max=255).astype(int)

obj_mask = volume >= len(images)

data = {
    "version": "Voxel Builder 4.5.1",
    "project": {
        "name": "naruto",
        "voxels": int(np.sum(obj_mask))
    },
    "data": {
        "voxels": ";".join([f"{v[0]},{v[1]},{v[2]},{rgb_to_hex(intensities[v[0], v[1], v[2]])},1" for v in zip(*np.where(obj_mask))])
    }
}
with open('shaded_model.json', 'w') as fp:
    json.dump(data, fp)

## Computing Reconstruction Error

In [ ]:
# Voxelize the ground truth mesh
voxel_size = step[0]  # Assuming uniform voxel size in all dimensions
gt_volume = mesh.voxelized(pitch=voxel_size)
gt_volume_dense = gt_volume.matrix.astype(bool)

# Align the ground truth volume to the reconstruction volume
# This may require adjusting the origin and orientation
gt_volume_dense_aligned = align_volumes(gt_volume_dense, obj_mask)


In [ ]:
gt_volume_dense.shape